In [1]:
import os
import pandas as pd

In [2]:
def clean_clm_name_dframe(dframe):
    dframe.columns = [i.lower().strip().replace(' ','_') for i in dframe.columns]
    return dframe

In [7]:
df=clean_clm_name_dframe(pd.read_csv('all_data.csv'))

In [9]:
# Split the Product Name field into Product Type and Size
df[['product_name','size']]=df['product_name'].str.split(' - ',expand=True)
df

In [19]:
# Only keep the Liquid products
df=df.loc[df['product_name']=='Liquid']

In [25]:
# Total up the sales for each Product Size and Scent for each Store
df=df[['sale_value', 'product_name', 'store_name','scent_name', 'size']]
df=df.groupby(['product_name', 'store_name','scent_name', 'size']).sum().reset_index().set_index(['store_name','scent_name', 'size']).sort_values(['sale_value'],ascending=False).reset_index()

In [62]:
df['rank']=df.groupby(['store_name'])['sale_value'].rank(ascending=False)

In [71]:
df=df.sort_values(['store_name','sale_value','rank'],ascending=False).reset_index(drop=True)

In [76]:
# Only leave the top 10 based on total sales value calculated above

df=df.loc[df['rank']<=10].reset_index(drop=True)


In [113]:
# Round the Sales Values to the nearest 10 value (ie 1913 becomes 1910)
def custom_round(x, base=5):
    return int(base * round(float(x)/base))
df['sale_value']=df['sale_value'].apply(lambda x: custom_round(x, base=5))



In [122]:
def display_stores(df):
    store_dict = {}
    counter = 1
    for i in df['store_name'].unique():
        store_dict[counter]=i
        print(str(counter),':',i)
        counter+=1
    store_dict[counter]='Exit'
    print(f'{counter} : Exit')
    return store_dict

In [124]:
stores = display_stores(df)
selected_store=int(input(f'Select store: 1-{len(stores)}'))

df.loc[df['store_name']==stores[selected_store]].to_csv(f'{stores[selected_store]}.csv',index=False)
os.startfile(f'{stores[selected_store]}.csv')

1 : Wimbledon
2 : Shoreditch
3 : Notting Hill
4 : Lewisham
5 : Dulwich
6 : Chelsea
7 : Exit
